# **Patching Surface for *snappyHexMesh* Generation**
<hr>

In [1]:
from vmtk import vmtkscripts
import numpy as np
import os

The default current directory when loading a notebook from GDrive is the home folder of the current user. To load the user's libraries and modules, first change to the file location using the os' python module, and load the necessary modules, then change to the former directory. 

In [2]:
# Change to directory where vmtk_functions and vmtk_filenames are located
os.chdir('/home/iagolessa/Documents/aneurysms/vmtkScripts')

# Import the necessary user's modules
import vmtk_functions
from vmtk_filenames import *

# Change it back
os.chdir('/home/iagolessa')

<hr>
# **Surface Orientation and Patching**

The cells below takes care of orienting the surface by translate and rotate transformations, to conform the inlet patch surface normal to the opposite z direction (this orientation was chosen to facilitate the introduction of boundary conditions at simulation time). 

The oriented surface is then 'exploded' to its constituent patches using the 'vmtkthreshold' script. First, the boundary is inspected generating a reference system file -- 'referenceSystem.dat' with the normal and center information of each cap patch (the wall has id 1 by default, and we prefer to keep it that way). The capped surface is, then, transformed and saved to disk -- 'surfaceOriented.stl' -- and finally is decomposed into its patches.

One of the cells below try to identify the inlet patch index by the patch area, because, usually, the inlet has larger area than the outlets (but is always important to be sure of the inlet id).

In [3]:
# Path to save the generated files when extracting the surface
# We reccomend to separate it from the DICOM original dir. 
# Note that inside casePath, this code will create different sub-directories to store different data types
casePath       = '/home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/'

# Define subdirs
imagesDir       = casePath+'images/'
surfacesDir     = casePath+'surfaces/'
centerlinesDir  = casePath+'centerlines/'
meshesDir       = casePath+'meshes/'
parentVesselDir = casePath+"parentVessel/"

print('Files saved to:', casePath)

Files saved to: /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/


In [4]:
# Get array with pacthes info
# surface = vmtk_functions.readSurface(surfacesDir+'surfaceRemeshedSpatialVaryingResolution.stl')
surface = vmtk_functions.readSurface(meshesDir+'wallSurfaceMesh.vtp')
# capsGeometryArray = vmtk_functions.getPatchInfo(surface)

Reading VTK XML surface file.


In [9]:
colsType = [('Center', tuple),
            ('Normal', tuple),
            ('Radius', float),
            ('PatchId', int),
            ('PatchType','U10')]

# Empty auxiliar list     
capsGeometryList = []

# Capping the surface
surfaceCapper = vmtkscripts.vmtkSurfaceCapper()
surfaceCapper.Surface = surface
surfaceCapper.Method = 'centerpoint'
surfaceCapper.Interactive = 0
surfaceCapper.Execute()

# Surface boundary inspector
# Conveting surface to mesh to get cell entity ids
surfaceToMesh = vmtkscripts.vmtkSurfaceToMesh()
surfaceToMesh.Surface = surfaceCapper.Surface
surfaceToMesh.Execute()
surfaceToMesh.PrintOutputMembers()

In [11]:
meshViewer = vmtkscripts.vmtkMeshViewer()
meshViewer.Mesh = surfaceToMesh.Mesh
meshViewer.Execute()

Quit renderer


In [ ]:
# Inspecting
surfaceBoundaryInspector = vmtkscripts.vmtkMeshBoundaryInspector()
surfaceBoundaryInspector.Mesh = surfaceToMesh.Mesh
surfaceBoundaryInspector.CellEntityIdsArrayName = surfaceCapper.CellEntityIdsArrayName
surfaceBoundaryInspector.Execute()

# Store patch info in python dictionary using vmtksurfacetonumpy
vmtkToNumpy = vmtkscripts.vmtkSurfaceToNumpy()
vmtkToNumpy.Surface = surfaceBoundaryInspector.ReferenceSystems
vmtkToNumpy.Execute()

In [ ]:
dictPatchData = vmtkToNumpy.ArrayDict
# Creation of a 'capsGeometryArray' ndarray structured object
# which contains Center, Normals, Radius, and patch type
# of the surface caps

# Assigning structuired array with patch info
intPatchesNum = len( dictPatchData['CellData']['CellPointIds'] )

for index in range(intPatchesNum):
    # Copy formatted to list
    Center  = tuple(dictPatchData['Points'][index])
    Normal  = tuple(dictPatchData['PointData']['BoundaryNormals'][index])
    PatchId = dictPatchData['PointData']['CellEntityIds'][index]
    Radius  = dictPatchData['PointData']['BoundaryRadius'][index]

    capsGeometryList.append((Center, Normal, Radius, PatchId, 'patch'))

# Convert to array
capsGeometryArray = np.array(capsGeometryList, 
                             dtype=colsType)

# Change patch type based on maximum radius
maxRadius = np.max(capsGeometryArray['Radius'])

for i in range(0, len(capsGeometryArray)):
    if  capsGeometryArray['Radius'][i] == maxRadius:
        capsGeometryArray['PatchType'][i] = 'inlet'
    else:
        capsGeometryArray['PatchType'][i] = 'outlet'

In [50]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Specify the other inlets here
extraInletIds = []

# For all the patches ids
for patchId in capsGeometryArray['PatchId']:
    # If the id is an extraInlet id
    # Check if extraInletIds is not empty
    if extraInletIds:
        # If the patch id is on specified inlet list
        if patchId in extraInletIds:
            # find its index in capsGeometryArray
            index = np.where(capsGeometryArray['PatchId'] == patchId)
            capsGeometryArray['PatchType'][index[0]] = 'inlet'
        # If not, then name it 'outlet'
        else:
            index = np.where(capsGeometryArray['PatchId'] == patchId)
            capsGeometryArray['PatchType'][index[0]] = 'outlet'
    else:
        continue
        
print(capsGeometryArray)

[ ((1.2600814, -9.519228, 14.469291), (0.058484861738628026, -0.39326909998835574, 0.91756140717761348),  0.95065012, 2, 'outlet')
 ((-0.0093680862, 0.014941562, -0.013540019), (8.1287364722144029e-08, -7.0293755045480791e-08, -0.99999999999999423),  1.4377415 , 3, 'inlet')
 ((11.146332, -12.368683, 3.232595), (0.83306690314413401, -0.45709888558626527, 0.31154476994749358),  1.1293027 , 4, 'outlet')]


In [51]:
# Last index of patches: to be used below when iterating over patches

# --> Volume is always 0
# --> Wall is always 1
wallIndex = surfaceBoundaryInspector.WallCellEntityId # = 1

# Inlet index: get inlet line first in array
inletPos = np.where(capsGeometryArray['PatchType'] == 'inlet')
inletId = int(capsGeometryArray['PatchId'][inletPos])

lastId = np.max(dictPatchData['PointData']['CellEntityIds'])
print('Last numbered patch index: ', lastId)

Last numbered patch index:  4


In [52]:
# Counting the number of inlets and outlets
# using the numpy.unique function: it returns an ndarray with the unique itens of the input array
# with the return_counts=True also returns the number of appearance of each item
patchTypes, numberOfPatches = np.unique(capsGeometryArray['PatchType'], return_counts=True)

# Zip both patchTypes and numberOfPatches and convert to dictionary
# with number of inlets and outlets
nPatches = dict(zip(patchTypes, numberOfPatches))
nPatches

{'inlet': 1, 'outlet': 2}

In [53]:
# Surface reorientation
# It is better to reorient the final capped surface before sending to mesh in SnappyHexMesh
# The vmtksurfacereferencesystemtransform script takes a surface (the surfaceCapped above) 
# and rotates and translates it conforming one of its patches (in our case the inlet patch) to a target reference system
# for that, it uses the output of the vtmkboundaryinspector

surfaceTransform = vmtkscripts.vmtkSurfaceReferenceSystemTransform()

# Load from file
# surfaceTransform.SurfaceInputFileName = surfacesDir+'surfaceCapped.vtp'
# surfaceTransform.IORead()

# or use the just created surfaceCapped
surfaceTransform.Surface = surfaceToMesh.Surface

# Target reference system parameters
surfaceTransform.ReferenceOrigin  = [0, 0, 0]  # translate to origin of system
surfaceTransform.ReferenceNormal1 = [0, 0, -1] # inlet normal will coincide with -z axis orientation
surfaceTransform.ReferenceNormal2 = [0, 0, -1]

# Surface reference system
surfaceTransform.ReferenceSystems = surfaceBoundaryInspector.ReferenceSystems

# to get the reference systems of inlet patch
# Note that, if there is more than one inlet, the inlet chose is the one with largest area
surfaceTransform.ReferenceSystemId = inletId

surfaceTransform.ReferenceSystemsIdArrayName = 'CellEntityIds'
surfaceTransform.ReferenceSystemsNormal1ArrayName = 'BoundaryNormals'
surfaceTransform.ReferenceSystemsNormal2ArrayName = 'BoundaryNormals'

# NEEDS TO BE A VTP FILE TO SAVE ARRAY DATA!
surfaceTransform.SurfaceOutputFileName = surfacesDir+surfaceOrientedFile

surfaceTransform.PrintInputMembers()
surfaceTransform.PrintOutputMembers()

# surfaceTransform.OutputText('Transforming capped surface. \n')
surfaceTransform.Execute()
surfaceTransform.IOWrite()
# surfaceTransform.OutputText('File saved: '+surfaceTransform.SurfaceOutputFileName)

Input vmtksurfacereferencesystemtransform members:
    Id = 0
    Disabled = 0
    Surface = vtkPolyData
    SurfaceInputFileName = 
    ReferenceOrigin = [0, 0, 0]
    ReferenceNormal1 = [0, 0, -1]
    ReferenceNormal2 = [0, 0, -1]
    ReferenceSystems = vtkPolyData
    ReferenceSystemsInputFileName = 
    ReferenceSystemId = 3
    ReferenceSystemsIdArrayName = CellEntityIds
    ReferenceSystemsNormal1ArrayName = BoundaryNormals
    ReferenceSystemsNormal2ArrayName = BoundaryNormals
    SurfaceOutputFileName = /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/surfaces/surfaceOriented.vtp
Output vmtksurfacereferencesystemtransform members:
    Id = 0
    Surface = vtkPolyData
Writing VTK XML surface file.


In [54]:
# Check if patch dir exists
# If it not exists, create it
# the patch dir will contains the surface patch files for meshing (wall, inlet and outlet(s)) 

patchDir = surfacesDir+'patches/'

if not os.path.isdir(patchDir):
    os.makedirs(patchDir)
    print(patchDir+' created.')

In [55]:
# Using vmtkThreshold script to extract patches for mesh generations in snappy

# Surface name to decompose if needs to be read before
# if you uncomment these next two lines, dont forget to change line 22 below
# surfaceOriented = readSurface(surfacesDir+surfaceCappedFile)
# print('\n')

# File names for patches
wallFileName = patchDir+'wall.stl'

# Extracting first the wall
extractThreshold = vmtkscripts.vmtkThreshold()

extractThreshold.Surface = surfaceTransform.Surface
extractThreshold.LowThreshold = wallIndex
extractThreshold.HighThreshold = wallIndex
extractThreshold.SurfaceOutputFileName = wallFileName
print('Extracting surface with id ', wallIndex)
extractThreshold.Execute()
extractThreshold.IOWrite()
extractThreshold.OutputText('Patch saved in '+extractThreshold.SurfaceOutputFileName+'\n')

# Outlet initial index (to increment)
outletIndex = 1
inletIndex = 1

# Loop to extract inlet and outlet patches
for i in np.arange( len(capsGeometryArray) ):
    print('Extracting surface with id ', capsGeometryArray['PatchId'][i])
    # Instantiate vmtkthreshold
    extractThreshold = vmtkscripts.vmtkThreshold()
    extractThreshold.Surface = surfaceTransform.Surface
    extractThreshold.LowThreshold = capsGeometryArray['PatchId'][i]
    extractThreshold.HighThreshold = capsGeometryArray['PatchId'][i]
    
    # Defining output file names
    if capsGeometryArray['PatchType'][i] == 'inlet':
        #
        if nPatches['inlet'] == 1:
            inletFileName = patchDir+'inlet.stl'
        else:
            inletFileName = patchDir+'inlet'+str(inletIndex)+'.stl'
            inletIndex += 1
        #
        # Attribute surface output file name
        print('Patch with id '+str(capsGeometryArray['PatchId'][i])+' is an inlet. Defining file name: '+inletFileName) # debugging
        extractThreshold.SurfaceOutputFileName = inletFileName
        
    elif capsGeometryArray['PatchType'][i] == 'outlet':
        if nPatches['outlet'] == 1:
            outletFileName = patchDir+'outlet.stl'
        else:
            outletFileName = patchDir+'outlet'+str(outletIndex)+'.stl'
            outletIndex += 1
        print('Patch with id '+str(capsGeometryArray['PatchId'][i])+' is an outlet. Defining file name: '+outletFileName) # debugging
        extractThreshold.SurfaceOutputFileName = outletFileName
    
    extractThreshold.Execute()
    # Writing surface
    extractThreshold.OutputText('Patch saved in '+extractThreshold.SurfaceOutputFileName+'\n')
    extractThreshold.IOWrite()
    print('\n')

Extracting surface with id  1
Writing STL surface file.
Patch saved in /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/surfaces/patches/wall.stl
Extracting surface with id  2
Patch with id 2 is an outlet. Defining file name: /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/surfaces/patches/outlet1.stl
Patch saved in /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/surfaces/patches/outlet1.stl
Writing STL surface file.


Extracting surface with id  3
Patch with id 3 is an inlet. Defining file name: /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/surfaces/patches/inlet.stl
Patch saved in /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/surfaces/patches/inlet.stl
Writing STL surface file.


Extracting surface with id  4
P

<hr>
The two cells below save the oriented surface and its Voronoi diagram to the parent vessel reconstruction procedure 

In [16]:
# Final surface for parent vessel reconstruction
caseId = 'case11'

if not os.path.isdir(parentVesselDir+caseId):
    os.makedirs(parentVesselDir+caseId)
    
surfaceForParentVessel = parentVesselDir+caseId+"/"+caseId+"_model.vtp"
print(surfaceForParentVessel)
vmtk_functions.writeSurface(surfaceTransform.Surface, surfaceForParentVessel, "binary")

/home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/parentVessel/case11/case11_model.vtp
Writing VTK XML surface file.


In [17]:
# Compute Voronoi Diagram Separately for parent vessel reconstruction
voronoiDiagram = vmtkscripts.vmtkDelaunayVoronoi()

voronoiDiagram.Surface = surfaceTransform.Surface
# voronoiDiagram.SurfaceInputFileName = surfaceInputFile #casePath+"surfaces/surfaceRemeshed.vtp" 
# voronoiDiagram.IORead()
voronoiDiagram.CheckNonManifold = 1
voronoiDiagram.RemoveSubresolutionTetrahedra = 1
# voronoiDiagram.DelaunayTessellationOutputFileName = casePath+"surfaces/delaunayTesselation.vtp"
voronoiDiagram.VoronoiDiagramOutputFileName = parentVesselDir+caseId+"/"+caseId+"_voronoi.vtp"
voronoiDiagram.Execute()
voronoiDiagram.IOWrite()

NonManifold check.
Cleaning surface.
Triangulating surface.
Writing VTK XML surface file.


In [19]:
vmtk_functions.viewSurface(voronoiDiagram.Surface)

Quit renderer
